# The Sure Tomorrow Insurance Company

The Sure Tomorrow insurance company wants to solve several tasks with the help of Machine Learning and we are asked to evaluate that possibility.

- Task 1: Find customers who are similar to a given customer. This will help the company's agents with marketing.
- Task 2: Predict whether a new customer is likely to receive an insurance benefit. Can a prediction model do better than a dummy model?
- Task 3: Predict the number of insurance benefits a new customer is likely to receive using a linear regression model.
- Task 4: Protect clients' personal data without breaking the model from the previous task. It's necessary to develop a data transformation algorithm that would make it hard to recover personal information if the data fell into the wrong hands. This is called data masking, or data obfuscation. But the data should be protected in such a way that the quality of machine learning models doesn't suffer. You don't need to pick the best model, just prove that the algorithm works correctly.

# Data Preprocessing & Exploration

## Initialization

In [44]:
# pip install scikit-learn --upgrade

In [45]:
# import libraries
import numpy as np
import pandas as pd
import math
import seaborn as sns
import sklearn.linear_model
import sklearn.metrics
from sklearn.metrics import f1_score
import sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
import sklearn.preprocessing
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split, cross_val_score
from IPython.display import display
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, r2_score, mean_squared_error as mse
from sklearn.utils import shuffle
import plotly_express as px
import plotly.graph_objects as go 
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors

## Load Data

Load data and conduct a basic check that it's free from obvious issues.

In [46]:
# read dataframe
df = pd.read_csv('datasets/insurance_us.csv')

We rename the colums to make the code look more consistent with its style.

In [47]:
# change column names
df = df.rename(columns={'Gender': 'gender', 'Age': 'age', 'Salary': 'income', 'Family members': 'family_members', 'Insurance benefits': 'insurance_benefits'})

In [48]:
# look at the data
df.sample(10)

We have 4 features, gender, age, salary, and family members, and one target, insurance benefits

In [49]:
# info on columns
df.info()

In [50]:
# we may want to fix the age type (from float to int) though this is not critical

# write your conversion here if you choose:
df.age = df.age.astype('int')
df.income = df.income.astype('int')

In [51]:
# check to see that the conversion was successful
df.info()

In [52]:
# looking for missing values
df.isna().sum()

In [53]:
# looking for duplicates
df[['age','gender', 'income', 'family_members', 'insurance_benefits']].duplicated().sum()

In [54]:
# now have a look at the data's descriptive statistics. 
# Does everything look okay?
df.describe()

### Conclusions

The data is clean, with no missing values. The duplicates that we see may not be true duplicates. Without unique identifiers, we assume that these are all unique values. 

## EDA

In [55]:
# correlations 
df.corr()

In [56]:
# data skew
df.skew()

In [57]:
# correlation matrix
px.imshow(df.corr(), title='Correlation Matrix', text_auto=True, height=900, 
    template='ggplot2')

Let's quickly check whether there are certain groups of customers by looking at the pair plot.

In [58]:
# scatter matrix
fig = px.scatter_matrix(df, height=800, title='Scatter Matrix')
fig.update_traces(showupperhalf=False, diagonal_visible=False)

Ok, it is a bit difficult to spot obvious groups (clusters) as it is difficult to combine several variables simultaneously (to analyze multivariate distributions). That's where LA and ML can be quite handy. Correlation matrix shows there is a positive relationship between age and insurance benefits. All other features show weak relationships. 

In [59]:
# distribution of age
px.histogram(df.age, title=' Distribution of Age', template='ggplot2', height=800, labels={'value': 'Age'})

Age shows a right skew distribution, with the mean and median at around 30. The minimum age is 18, while the maximum age is 65 years old.

In [60]:
# distribution of income
px.histogram(df.income, title='Distribution of Income', template='plotly_dark', height=800, labels={'value': 'Salary'})

Income is normally distributed around the mean of $40,000. Income rages from $5,300 to $79,000. 

In [61]:
# distribution of family members
px.histogram(df.family_members, title='Distribution of Family Members', template='seaborn', height=800, labels={'value': 'Number of Family Members'})

The distribution of family members is skewed to the right. The range of values is 0 to 6 with a mean of 1.19, and a median of 1.0. 

In [62]:
# distribution of gender
px.bar(df.gender.value_counts(), color_discrete_sequence=[['pink', 'blue']], labels={'index': 'Gender', 'value': 'Count'}, title='Distribution of Gender', height=800)

Gender is more or less identical in the dataset. We assume 0 is female, and male is 1. 

In [63]:
# distribution of insurance benefits
px.histogram(df.insurance_benefits,  labels={'value': 'Number of Benefits'}, title='Distribution of Insurance Benefits', height=800, template='none')

The distribution of insurance benefits is heavily skewed to the right. The mean is 0.14, and the median is 0. The maximum value is 5.  

# Task 1. Similar Customers

In [64]:
# euclidean
feature_names = ['gender', 'age', 'income', 'family_members']
nbrs = NearestNeighbors(metric='euclidean')
nbrs.fit(df[feature_names])
nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[1][feature_names]], n_neighbors=5, return_distance=True)
df_res = pd.concat([df.iloc[nbrs_indices[0]], pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])], axis=1)

In [65]:
# euclidean
df_res.head()

In [66]:
# Manhattan 
feature_names = ['gender', 'age', 'income', 'family_members']
nbrs = NearestNeighbors(metric='manhattan')
nbrs.fit(df[feature_names])
nbrs_distances, nbrs_indices = nbrs.kneighbors([df.iloc[1][feature_names]], n_neighbors=5, return_distance=True)
df_res = pd.concat([df.iloc[nbrs_indices[0]], pd.DataFrame(nbrs_distances.T, index=nbrs_indices[0], columns=['distance'])], axis=1)


In [67]:
# manhattan
df_res.head()

# The Sure Tomorrow Insurance Company

Scaling the data.

In [68]:
# scaling numerical columns
feature_names = ['gender', 'age', 'income', 'family_members']

transformer_mas = sklearn.preprocessing.MaxAbsScaler().fit(df[feature_names].to_numpy())

df_scaled = df.copy()
df_scaled.loc[:, feature_names] = transformer_mas.transform(df[feature_names].to_numpy())

In [69]:
# look at scaled data
df_scaled.sample(5)

Now, let's get similar records for a given one for every combination

In [70]:
# euclidean
feature_names = ['gender', 'age', 'income', 'family_members']
nbrs2 = NearestNeighbors(metric='euclidean')
nbrs2.fit(df_scaled[feature_names])
nbrs_distances2, nbrs_indices2 = nbrs2.kneighbors([df_scaled.iloc[1][feature_names]], n_neighbors=5, return_distance=True)
df_res2 = pd.concat([df_scaled.iloc[nbrs_indices2[0]], pd.DataFrame(nbrs_distances2.T, index=nbrs_indices2[0], columns=['distance'])], axis=1)

In [71]:
# euclidean
df_res2.head()

In [72]:
# manhattan
feature_names = ['gender', 'age', 'income', 'family_members']
nbrs2 = NearestNeighbors(metric='manhattan')
nbrs2.fit(df_scaled[feature_names])
nbrs_distances2, nbrs_indices2 = nbrs2.kneighbors([df_scaled.iloc[1][feature_names]], n_neighbors=5, return_distance=True)
df_res2 = pd.concat([df_scaled.iloc[nbrs_indices2[0]], pd.DataFrame(nbrs_distances2.T, index=nbrs_indices2[0], columns=['distance'])], axis=1)

In [73]:
# manhattan
df_res2.head()

**Does the data being not scaled affect the kNN algorithm? If so, how does that appear?** 

When comparing the unscaled results of euclidean and manhattan metrics, we see differences in results. However, after scaling the data, the results of the two metrics are the same. Therefore, scaling does affect the results. Scaling levels the income values, which is magnitudes greater than the other features. 

**How similar are the results using the Manhattan distance metric (regardless of the scaling)?** 

The manhattan results are the euclidean results rounded up to the next integer.

# Task 2. Is Customer Likely to Receive Insurance Benefit?

## Unscaled

In [74]:
# look at data
df.head()

In [75]:
# binarize target with threshold of 0.5, drop old target column
binarizer = Binarizer(threshold=0.5)
df['insurance_benefits_received'] = binarizer.fit_transform(df[['insurance_benefits']])

In [76]:
# probability of insurance benefit received
df['insurance_benefits_received'].sum() / len(df)

In [77]:
# check for the class imbalance with value_counts()
df.insurance_benefits_received.value_counts()

In [78]:
# unscaled data
X = df[['age', 'gender', 'income', 'family_members']]
y = df['insurance_benefits_received']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)

In [79]:
# unscaled knn model
neighbors = np.arange(1,11)
test_accuracies = {}

for neighbor in neighbors:
    knn = KNeighborsClassifier(n_neighbors=neighbor)
    knn.fit(X_train, y_train)
    predicted_test = knn.predict(X_test)
    test_accuracies[neighbor] = f1_score(y_test, predicted_test)
    
print(neighbors, '\n', test_accuracies)

In [80]:
# creating a dataframe for accuracies, transposed
accuracies = pd.DataFrame([test_accuracies.values()], columns=test_accuracies.keys()).T

In [81]:
# grapph of train and test accuracies
px.line(accuracies, title='KNN: Varying Number of Neighbors', labels={'index': 'Number of Neighbors', 'value': 'F1_score'})

### Conclusions 

The model parameter that gave the best results was 3 nearest neighbors, then the test score general trend is down. 

## Scaled

In [82]:
# binarize target with threshold of 0.5, drop old target column
binarizer = Binarizer(threshold=0.5)
df_scaled['insurance_benefits_received'] = binarizer.fit_transform(df_scaled[['insurance_benefits']])

In [83]:
# scaled data
X_scaled = df_scaled[['age', 'gender', 'income', 'family_members']]
y_scaled = df_scaled['insurance_benefits_received']

X_scaled_train, X_scaled_test, y_scaled_train, y_scaled_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=19)

In [84]:
# unscaled knn model
neighbors = np.arange(1,11)

test_accuracies = {}

for neighbor in neighbors:
    knn_scaled = KNeighborsClassifier(n_neighbors=neighbor)
    knn_scaled.fit(X_scaled_train, y_scaled_train)
    predicted_test_scaled = knn_scaled.predict(X_scaled_test)
    test_accuracies[neighbor] = f1_score(y_scaled_test, predicted_test_scaled)
    
print(neighbors, '\n', test_accuracies)

In [85]:
# creating a dataframe for accuracies, transposed
accuracies = pd.DataFrame([ test_accuracies.values()], columns=test_accuracies.keys()).T

In [86]:
# grapph of train and test accuracies
px.line(accuracies, title='KNN: Varying Number of Neighbors', labels={'index': 'Number of Neighbors', 'value': 'F1 Score'})

### Conclusions 

The model parameter that gave the best results was 5 nearest neighbors. Then, the test scores fluctuate, but the general trend is down. These results are better than the results of the unscaled data, for each respective amount of neighbors. 

## Dummy Model

In [87]:
# function for classifier evaluation
def eval_classifier(y_true, y_pred):
    
    f1_score = sklearn.metrics.f1_score(y_true, y_pred)
    print(f'F1: {f1_score:.2f}')
    
# if you have an issue with the following line, restart the kernel and run the notebook again
    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='all')
    print('Confusion Matrix')
    print(cm)

In [88]:
# generating output of a random model

def rnd_model_predict(P, size, seed=42):

    rng = np.random.default_rng(seed=seed)
    return rng.binomial(n=1, p=P, size=size)

In [89]:
# probabilities
for P in [0, df['insurance_benefits_received'].sum() / len(df), 0.5, 1]:

    print(f'The probability: {P:.2f}')
    y_pred_rnd = rnd_model_predict(P, 5000)
        
    eval_classifier(df['insurance_benefits_received'], y_pred_rnd)
    
    print()

### Conclusion

The dummy model demonstrates F1 scores of different probabilities for insurance benefits received. We see that the nearest neighbor models with the scaled and unscaled data have better f1 scores than the dummy model. Therefore, the previous models are better at classifying insurance benefits received than a random model. 

# Task 3. Regression (with Linear Regression)

In [90]:
# creating linear regression algorithm 
class MyLinearRegression:
    
    def __init__(self):
        
        self.weights = None
    
    def fit(self, X, y):
        
        # adding the unities
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        self.weights = np.linalg.inv(X2.T @ X2) @ X2.T @ y

    def predict(self, X):
        
        # adding the unities
        X2 = np.append(np.ones([len(X), 1]), X, axis=1)
        y_pred = X2 @ self.weights
        
        return y_pred

In [91]:
# evaluation for regressor algorithm
def eval_regressor(y_true, y_pred):
    
    rmse = math.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    print(f'RMSE: {rmse:.2f}')
    
    r2_score = math.sqrt(sklearn.metrics.r2_score(y_true, y_pred))
    print(f'R2: {r2_score:.2f}')    

## Original data

In [92]:
# Running regression model on original data
X = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

lr = MyLinearRegression()

lr.fit(X_train, y_train)
print(lr.weights)

y_test_pred = lr.predict(X_test)
eval_regressor(y_test, y_test_pred)

## Scaled Data

In [93]:
# Running regression model on scaled data
X_scaled = df_scaled[['age', 'gender', 'income', 'family_members']].to_numpy()
y_scaled = df_scaled['insurance_benefits'].to_numpy()

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=12345)

lr_scaled = MyLinearRegression()

lr_scaled.fit(X_train_scaled, y_train_scaled)
print(lr_scaled.weights)

y_test_pred_scaled = lr_scaled.predict(X_test_scaled)
eval_regressor(y_test_scaled, y_test_pred_scaled)

### Conclusions

Running a linear regression on both the original data and the scaled data, we see how the evaluation metrics do not change. The RMSE is 0.34, and the R2 is 0.66. Therefore, scaling the data did not change the accuracy of the model. 

# Task 4. Obfuscating Data

In [94]:
# data to be obfuscated
personal_info_column_list = ['gender', 'age', 'income', 'family_members']
df_pn = df[personal_info_column_list]

In [95]:
# convert data to numpy array
X = df_pn.to_numpy()

In [96]:
# look at the array
X

Generating a random matrix $P$.

In [97]:
# random factor P
rng = np.random.default_rng(seed=42)
P = rng.random(size=(X.shape[1], X.shape[1]))

Checking the matrix $P$ is invertible

In [98]:
# visual of P
P

In [99]:
# inverse of P
P_inv = np.linalg.inv(P)
P_inv

In [100]:
# new obfuscated dataset
X_new = X @ P
X_new

Can you guess the customers' ages or income after the transformation?

No

Can you recover the original data from $X'$ if you know $P$?

In [101]:
# product of P and X
P @ X.T

In [117]:
# product of inverse of P and X_new
X_new @ P_inv

Print all three cases for a few customers
- The original data
- The transformed one
- The reversed (recovered) one

In [103]:
# original 
X

In [104]:
# transformed
X @ P

In [105]:
# recovered 
X_recovered = P_inv @ X_new.T
X_recovered

You can probably see that some values are not exactly the same as they are in the original data. What might be the reason for that?

## Test Linear Regression With Data Obfuscation

## Original

In [106]:
# original data
X = df[['age', 'gender', 'income', 'family_members']].to_numpy()
y = df['insurance_benefits'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)

linreg = MyLinearRegression()

linreg.fit(X_train, y_train)
print(linreg.weights)

y_test_pred = linreg.predict(X_test)
eval_regressor(y_test, y_test_pred)

## Obfuscated

In [3]:
# Obfuscated data
# X_new
# y = df['insurance_benefits'].to_numpy()

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_new, y, test_size=0.3, random_state=19)

linreg2 = MyLinearRegression()

linreg2.fit(X_train2, y_train2)
print(linreg2.weights)

y_test_pred2 = linreg2.predict(X_test2)
eval_regressor(y_test2, y_test_pred2)

NameError: name 'X_new' is not defined

### Conclusion

Running a linear regression on both the original data and the obfuscated data, we see how the evaluation metrics do not change. The RMSE is 0.37, and the R2 is 0.66. Therefore, obfuscating the data did not break the model, as the accuracy did not change. 

# Final Conclusions

We trained a model that would return similar customers for a given one. This model was calculated while scaled and unscaled, using euclidean and manhattan distances. Then, we created a dummy model to test the f1 scores of different probability values. We found the dummy model to be less accurate than the classification model we built, using both original and scaled data. After, a linear regression model was built with matrix operations. The evaluation metrics of RMSE and R2 score were measured, and then compared to a linear regression model on the obfuscated data. We concluded that obfuscation did not alter the accuracy of the model, as the RMSE and R2 metrics were the same before and after obfuscation. Overall, we have provided results that suggest a very accurate prediction as to whether a customer will, or will not receive insurance benefits. This is more accurate than trying to predict the actual number of insurance benefits a customer will receive. Consequently, we suggest Sure Tomorrow use the more accurate classification model over the regression model.  